In [3]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
#import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
mydata = "./data/"
zdata = "../zurita/data/"
fat = pd.read_csv(mydata + "faturamento_lojas_atuais.csv")
cen_exp = pd.read_csv(mydata + "cenarios_expansao.csv")

In [5]:
cen_exp

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.00,FEATURE_05_VALUE_03,6,0.134654,...,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8
1,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,41.00,FEATURE_05_VALUE_03,4,0.116662,...,26.345742,1.312612,NaN,NaN,84.3,77.5,100.0,100.0,63.5,78.8
2,16,2928703003003,2928703,NaN,FEATURE_02_VALUE_04,False,42.12,FEATURE_05_VALUE_05,2,0.104162,...,35.362788,1.969512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21,2906006003001,2906006,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,6,0.132954,...,9.862757,1.383681,NaN,NaN,85.0,76.9,100.0,100.0,100.0,92.9
4,28,3550308005058,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,44.31,FEATURE_05_VALUE_03,8,0.190102,...,5593.905463,2.815883,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,3667,2927408005009,2927408,FEATURE_01_VALUE_03,FEATURE_02_VALUE_03,True,11.47,FEATURE_05_VALUE_06,3,0.107963,...,1355.486603,4.142294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,3672,-1,-1,NaN,FEATURE_02_VALUE_03,False,31.00,FEATURE_05_VALUE_05,5,0.190896,...,32.409193,2.989030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,3674,5208707005001,5208707,NaN,FEATURE_02_VALUE_03,False,58.00,FEATURE_05_VALUE_05,9,0.190896,...,661.415873,2.795840,NaN,NaN,100.0,78.3,100.0,100.0,60.6,81.1
551,3675,4218202003002,4218202,NaN,FEATURE_02_VALUE_04,False,40.74,FEATURE_05_VALUE_03,4,0.133754,...,20.027909,2.422305,NaN,NaN,100.0,92.5,100.0,100.0,94.2,96.6


In [6]:
rec_imp = pd.read_csv(mydata + "receitas_cen_exp_imputadas.csv")
rec_imp

,cod_loja,mes,ano,receita
0,10,1,2016,6851.975507
1,10,1,2017,5858.744198
2,10,2,2016,4808.792464
3,10,2,2017,5003.799853
4,10,3,2016,5455.608116
...,...,...,...,...
13844,3353,12,2017,18499.087986
13845,3436,12,2017,18499.087986
13846,3484,12,2017,18499.087986
13847,3626,12,2017,18499.087986


In [7]:
ce_merge = cen_exp.merge(rec_imp, on='cod_loja')
ce_merge['datetime'] = pd.to_datetime(ce_merge.ano*100 + ce_merge.mes, format='%Y%m')

In [8]:
ce_merge

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,mes,ano,receita,datetime
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,93.4,100.0,100.0,71.7,88.8,1,2016,6851.975507,2016-01-01
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,93.4,100.0,100.0,71.7,88.8,1,2017,5858.744198,2017-01-01
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,93.4,100.0,100.0,71.7,88.8,2,2016,4808.792464,2016-02-01
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,93.4,100.0,100.0,71.7,88.8,2,2017,5003.799853,2017-02-01
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,93.4,100.0,100.0,71.7,88.8,3,2016,5455.608116,2016-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,False,29.0,FEATURE_05_VALUE_03,5,0.123648,...,77.0,100.0,100.0,50.0,99.7,85.6,10,2017,4883.151739,2017-10-01
13845,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,False,29.0,FEATURE_05_VALUE_03,5,0.123648,...,77.0,100.0,100.0,50.0,99.7,85.6,11,2016,6492.640000,2016-11-01
13846,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,False,29.0,FEATURE_05_VALUE_03,5,0.123648,...,77.0,100.0,100.0,50.0,99.7,85.6,11,2017,7006.633088,2017-11-01
13847,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,False,29.0,FEATURE_05_VALUE_03,5,0.123648,...,77.0,100.0,100.0,50.0,99.7,85.6,12,2016,10912.574857,2016-12-01


In [9]:
ce_merge["quarter"] = ce_merge["datetime"].dt.quarter
ce_merge["ano"] = ce_merge["datetime"].dt.year
ce_merge["mes"] = ce_merge["datetime"].dt.month
ce_merge["feature_03"] = ce_merge["feature_03"].apply(lambda x: 1 if x else 0)

In [10]:
df = ce_merge

In [11]:
df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_14,feature_15,feature_16,feature_17,feature_18,mes,ano,receita,datetime,quarter
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,93.4,100.0,100.0,71.7,88.8,1,2016,6851.975507,2016-01-01,1
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,93.4,100.0,100.0,71.7,88.8,1,2017,5858.744198,2017-01-01,1
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,93.4,100.0,100.0,71.7,88.8,2,2016,4808.792464,2016-02-01,1
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,93.4,100.0,100.0,71.7,88.8,2,2017,5003.799853,2017-02-01,1
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,93.4,100.0,100.0,71.7,88.8,3,2016,5455.608116,2016-03-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,100.0,50.0,99.7,85.6,10,2017,4883.151739,2017-10-01,4
13845,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,100.0,50.0,99.7,85.6,11,2016,6492.640000,2016-11-01,4
13846,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,100.0,50.0,99.7,85.6,11,2017,7006.633088,2017-11-01,4
13847,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,100.0,50.0,99.7,85.6,12,2016,10912.574857,2016-12-01,4


In [12]:
def categoriza_lojas(row):
    if row["receita"] <= 75000:
        return 0
    elif row["receita"] <= 2e5:
        return 10
    elif row["receita"] <= 4e5:
        return 20
    elif row["receita"] <= 6e5:
        return 30
    else:
        return 40
    
def obtem_categoria(cod_loja):
    return categorias_df[categorias_df["cod_loja"] == cod_loja]["categoria_loja"].values[0]


categorias_df = df.groupby(["cod_loja"], as_index=False).mean()[["cod_loja", "receita"]]
categorias_df["categoria_loja"] = categorias_df.apply(categoriza_lojas, axis=1)

df["categoria_loja"] = df["cod_loja"].apply(obtem_categoria)

In [13]:
df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_15,feature_16,feature_17,feature_18,mes,ano,receita,datetime,quarter,categoria_loja
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,100.0,71.7,88.8,1,2016,6851.975507,2016-01-01,1,0
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,100.0,71.7,88.8,1,2017,5858.744198,2017-01-01,1,0
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,100.0,71.7,88.8,2,2016,4808.792464,2016-02-01,1,0
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,100.0,71.7,88.8,2,2017,5003.799853,2017-02-01,1,0
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,100.0,100.0,71.7,88.8,3,2016,5455.608116,2016-03-01,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,50.0,99.7,85.6,10,2017,4883.151739,2017-10-01,4,0
13845,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,50.0,99.7,85.6,11,2016,6492.640000,2016-11-01,4,0
13846,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,50.0,99.7,85.6,11,2017,7006.633088,2017-11-01,4,0
13847,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,100.0,50.0,99.7,85.6,12,2016,10912.574857,2016-12-01,4,0


In [14]:
def get_mean_of_my_category(row):
    my_category_and_month = df[(df["categoria_loja"] == row["categoria_loja"]) & (df["mes"] == row["mes"])]
    
    return np.mean(my_category_and_month["receita"].values)

df["receita_minha_categoria"] = df.apply(get_mean_of_my_category, axis=1)

In [16]:
feriados_df = pd.read_csv(mydata + "feriados_exp.csv")
df = df.merge(feriados_df, left_on=['cod_loja', 'mes'], right_on=['cod_loja', 'mes'])
df.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_17,feature_18,mes,ano,receita,datetime,quarter,categoria_loja,receita_minha_categoria,cnt_feriados
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,1,2016,6851.975507,2016-01-01,1,0,8639.382665,1.0
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,1,2017,5858.744198,2017-01-01,1,0,8639.382665,1.0
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,2,2016,4808.792464,2016-02-01,1,0,6641.056621,0.0
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,2,2017,5003.799853,2017-02-01,1,0,6641.056621,0.0
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,3,2016,5455.608116,2016-03-01,1,0,8046.645276,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,10,2017,4883.151739,2017-10-01,4,0,8443.880845,1.0
13845,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,11,2016,6492.640000,2016-11-01,4,0,10435.131667,2.0
13846,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,11,2017,7006.633088,2017-11-01,4,0,10435.131667,2.0
13847,3684,2917607003002,2917607,NaN,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,12,2016,10912.574857,2016-12-01,4,0,19773.756863,1.0


In [18]:
has_na_cols = list()
for col in df.columns:
    if df[col].isnull().values.any():
        has_na_cols.append(col)
has_na_cols

for na_col in has_na_cols:
    if df[na_col].dtype == object:
        print(df[na_col].mode()[0])
        df[na_col].fillna(df[na_col].mode()[0], inplace=True)
    else:
        df[na_col].fillna(df[na_col].median(), inplace=True)

FEATURE_01_VALUE_10
FEATURE_11_VALUE_02
FEATURE_12_VALUE_01


In [19]:
df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_17,feature_18,mes,ano,receita,datetime,quarter,categoria_loja,receita_minha_categoria,cnt_feriados
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,1,2016,6851.975507,2016-01-01,1,0,8639.382665,1.0
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,1,2017,5858.744198,2017-01-01,1,0,8639.382665,1.0
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,2,2016,4808.792464,2016-02-01,1,0,6641.056621,0.0
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,2,2017,5003.799853,2017-02-01,1,0,6641.056621,0.0
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,71.7,88.8,3,2016,5455.608116,2016-03-01,1,0,8046.645276,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,10,2017,4883.151739,2017-10-01,4,0,8443.880845,1.0
13845,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,11,2016,6492.640000,2016-11-01,4,0,10435.131667,2.0
13846,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,11,2017,7006.633088,2017-11-01,4,0,10435.131667,2.0
13847,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,99.7,85.6,12,2016,10912.574857,2016-12-01,4,0,19773.756863,1.0


In [20]:
lqdf = df[(df["cod_loja"] == 10) & (df["quarter"] == 2) & (df["ano"] == 2016)] \
                          .sort_values("datetime")[["datetime", "receita"]]

In [21]:
df.cod_loja.unique().size

553

In [22]:
receita_ultimo_trimestre = []

for index, row in df.iterrows():
    if row["ano"] == 2016 and row["quarter"] == 1:
        receita_ultimo_trimestre.append([index, 0,0,0,0])
    # Há semestre anterior
    else:
        #ultimo trimestre do ano passado
        quarter = row["quarter"] - 1
        ano = row["ano"]
        
        if quarter == 0:
            ano = row["ano"]-1
            quarter = 4
            
        last_quarter_df = df[(df["cod_loja"] == row["cod_loja"]) & (df["quarter"] == quarter) & (df["ano"] == ano)] \
                          .sort_values("datetime")[["datetime", "receita"]]
        receita = last_quarter_df["receita"].values
        if len(receita) == 3:
            receita_ultimo_trimestre.append([index, receita[0], receita[1], receita[2], np.mean(receita)])
        else:
            ## rever
            receita_ultimo_trimestre.append([index, 0, 0, 0, 0])
            
    
receita_ultimo_trimestre_df = pd.DataFrame(receita_ultimo_trimestre, columns=["index", "receita_trim1", "receita_trim2", "receita_trim3", "receita_media"]).set_index("index")
receita_ultimo_trimestre_df.head()
df = df.merge(receita_ultimo_trimestre_df, left_index=True, right_index=True)

In [23]:
df.columns

Index(['cod_loja', 'cod_ap', 'cod_municipio', 'feature_01', 'feature_02',
       'feature_03', 'feature_04', 'feature_05', 'feature_06', 'feature_07',
       'feature_08', 'feature_09', 'feature_10', 'feature_11', 'feature_12',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'mes', 'ano', 'receita', 'datetime', 'quarter',
       'categoria_loja', 'receita_minha_categoria', 'cnt_feriados',
       'receita_trim1', 'receita_trim2', 'receita_trim3', 'receita_media'],
      dtype='object')

In [24]:
df.to_csv("./data/cen_exp_features_prontas.csv", index=False)

In [25]:
df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,receita,datetime,quarter,categoria_loja,receita_minha_categoria,cnt_feriados,receita_trim1,receita_trim2,receita_trim3,receita_media
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,6851.975507,2016-01-01,1,0,8639.382665,1.0,0.000000,0.00000,0.000000,0.000000
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,5858.744198,2017-01-01,1,0,8639.382665,1.0,6633.698235,7920.13500,11616.729268,8723.520835
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,4808.792464,2016-02-01,1,0,6641.056621,0.0,0.000000,0.00000,0.000000,0.000000
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,5003.799853,2017-02-01,1,0,6641.056621,0.0,6633.698235,7920.13500,11616.729268,8723.520835
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,FEATURE_05_VALUE_03,6,0.134654,...,5455.608116,2016-03-01,1,0,8046.645276,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,4883.151739,2017-10-01,4,0,8443.880845,1.0,4670.910429,6344.17250,5356.792286,5457.291738
13845,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,6492.640000,2016-11-01,4,0,10435.131667,2.0,4835.784000,6096.37942,5488.783134,5473.648852
13846,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,7006.633088,2017-11-01,4,0,10435.131667,2.0,4670.910429,6344.17250,5356.792286,5457.291738
13847,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,FEATURE_05_VALUE_03,5,0.123648,...,10912.574857,2016-12-01,4,0,19773.756863,1.0,4835.784000,6096.37942,5488.783134,5473.648852


In [52]:
dummy_df = pd.get_dummies(df, columns=["feature_05"])

In [53]:
dummy_df['feature_05_FEATURE_05_VALUE_08'] = 0

In [54]:
dummy_df.columns

Index(['cod_loja', 'cod_ap', 'cod_municipio', 'feature_01', 'feature_02',
       'feature_03', 'feature_04', 'feature_06', 'feature_07', 'feature_08',
       'feature_09', 'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
       'mes', 'ano', 'receita', 'datetime', 'quarter', 'categoria_loja',
       'receita_minha_categoria', 'cnt_feriados', 'receita_trim1',
       'receita_trim2', 'receita_trim3', 'receita_media',
       'feature_05_FEATURE_05_VALUE_01', 'feature_05_FEATURE_05_VALUE_02',
       'feature_05_FEATURE_05_VALUE_03', 'feature_05_FEATURE_05_VALUE_04',
       'feature_05_FEATURE_05_VALUE_05', 'feature_05_FEATURE_05_VALUE_06',
       'feature_05_FEATURE_05_VALUE_07', 'feature_05_FEATURE_05_VALUE_08'],
      dtype='object')

In [31]:
dummy_df.to_csv('./data/cen_expansao_features_prontas.csv')

In [55]:
dummy_df

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,...,receita_trim3,receita_media,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,6,0.134654,0.131481,...,0.000000,0.000000,0,0,1,0,0,0,0,0
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,6,0.134654,0.131481,...,11616.729268,8723.520835,0,0,1,0,0,0,0,0
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,6,0.134654,0.131481,...,0.000000,0.000000,0,0,1,0,0,0,0,0
3,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,6,0.134654,0.131481,...,11616.729268,8723.520835,0,0,1,0,0,0,0,0
4,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.0,6,0.134654,0.131481,...,0.000000,0.000000,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,5,0.123648,0.113905,...,5356.792286,5457.291738,0,0,1,0,0,0,0,0
13845,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,5,0.123648,0.113905,...,5488.783134,5473.648852,0,0,1,0,0,0,0,0
13846,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,5,0.123648,0.113905,...,5356.792286,5457.291738,0,0,1,0,0,0,0,0
13847,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.0,5,0.123648,0.113905,...,5488.783134,5473.648852,0,0,1,0,0,0,0,0


In [33]:
import xgboost as xgb
import pandas as pd
from datetime import date
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [287]:
submit_df = dummy_df

In [288]:
str_cols = []
str_cols.append("datetime")
str_cols.append("receita")

str_cols.append("feature_01")
str_cols.append("feature_02")
str_cols.append("feature_11")
str_cols.append("feature_12")
str_cols.append("receita_trim1")
str_cols.append("receita_trim2")
str_cols.append("receita_trim3")

In [289]:
submit_df.columns.size

40

In [290]:
pd.set_option("max_columns", None)

In [291]:
q4_2017 = submit_df[(submit_df['ano'] == 2017) & ((submit_df['mes'] == 10) | (submit_df['mes'] == 11) | (submit_df['mes'] == 12))]
linhas = submit_df.cod_loja
indices = linhas.drop_duplicates()
df_linhas = submit_df[submit_df.index.isin(indices.index)]

In [292]:
q4_2017

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,mes,ano,receita,datetime,quarter,categoria_loja,receita_minha_categoria,cnt_feriados,receita_trim1,receita_trim2,receita_trim3,receita_media,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08
19,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,10,2017,5666.608228,2017-10-01,4,0,8443.880845,1.0,4444.582195,6481.030886,5799.354691,5574.989258,0,0,1,0,0,0,0,0
21,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,11,2017,6868.386076,2017-11-01,4,0,10435.131667,2.0,4444.582195,6481.030886,5799.354691,5574.989258,0,0,1,0,0,0,0,0
23,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,12,2017,11020.644167,2017-12-01,4,0,19773.756863,1.0,4444.582195,6481.030886,5799.354691,5574.989258,0,0,1,0,0,0,0,0
43,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,10,2017,1437.750000,2017-10-01,4,0,8443.880845,1.0,1448.806400,1858.661600,1704.469231,1670.645744,0,0,1,0,0,0,0,0
45,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,11,2017,1837.855000,2017-11-01,4,0,10435.131667,2.0,1448.806400,1858.661600,1704.469231,1670.645744,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13822,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,11,2017,11713.015906,2017-11-01,4,0,10435.131667,3.0,7512.418278,10738.926689,7655.738859,8635.694609,0,0,1,0,0,0,0,0
13824,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,12,2017,23318.410473,2017-12-01,4,0,19773.756863,1.0,7512.418278,10738.926689,7655.738859,8635.694609,0,0,1,0,0,0,0,0
13844,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,10,2017,4883.151739,2017-10-01,4,0,8443.880845,1.0,4670.910429,6344.172500,5356.792286,5457.291738,0,0,1,0,0,0,0,0
13846,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,11,2017,7006.633088,2017-11-01,4,0,10435.131667,2.0,4670.910429,6344.172500,5356.792286,5457.291738,0,0,1,0,0,0,0,0


In [293]:
q4_2017    
q4_mean = q4_2017.groupby(['cod_loja'], as_index=False).mean()
rec_med_q4 = q4_mean[['cod_loja', 'receita']]

In [294]:
drop_cols = ['receita', 'mes', 'ano', 'quarter', 'datetime', 'receita_trim1', 'receita_trim2', 'receita_trim3', 'receita_media']
q4_drop = q4_2017.drop(columns=drop_cols)
q4_drop

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,categoria_loja,receita_minha_categoria,cnt_feriados,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08
19,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,0,8443.880845,1.0,0,0,1,0,0,0,0,0
21,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,0,10435.131667,2.0,0,0,1,0,0,0,0,0
23,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,0,19773.756863,1.0,0,0,1,0,0,0,0,0
43,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,0,8443.880845,1.0,0,0,1,0,0,0,0,0
45,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,0,10435.131667,2.0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13822,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,0,10435.131667,3.0,0,0,1,0,0,0,0,0
13824,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,0,19773.756863,1.0,0,0,1,0,0,0,0,0
13844,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,0,8443.880845,1.0,0,0,1,0,0,0,0,0
13846,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,0,10435.131667,2.0,0,0,1,0,0,0,0,0


In [295]:
datas = pd.DataFrame(data={'mes': [1, 2, 3], 'ano': [2018, 2018, 2018], 'quarter': [1, 1, 1]})

In [296]:
cod_lojas = q4_drop.cod_loja.unique().tolist() 
nums = [1, 2, 3]

In [297]:
idx = pd.MultiIndex.from_product([cod_lojas, nums], names=['cod_loja', 'mes'])

In [298]:
lojas_meses = pd.DataFrame(index = idx).reset_index()

In [299]:
lojas_meses['ano'] = 2018
lojas_meses['datetime'] = pd.to_datetime(lojas_meses.ano*100 + lojas_meses.mes, format='%Y%m')
lojas_meses["quarter"] = ce_merge["datetime"].dt.quarter

In [300]:
datas_rec = lojas_meses.merge(rec_med_q4, on='cod_loja')
datas_rec.rename(columns={'receita':'receita_media'}, inplace=True)

In [301]:
df_linhas.drop(columns=["mes", "ano", "datetime", "quarter", "receita_media"], inplace=True)

/usr/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [302]:
mergao = df_linhas.merge(datas_rec, on='cod_loja')
mergao

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,receita,categoria_loja,receita_minha_categoria,cnt_feriados,receita_trim1,receita_trim2,receita_trim3,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08,mes,ano,datetime,quarter,receita_media
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7851.879490
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7851.879490
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,1,7851.879490
3,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,2164.207949
4,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,2164.207949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,4,14337.009729
1655,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,4,14337.009729
1656,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7800.357442
1657,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,1.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7800.357442


In [303]:
mergao.drop(columns='cnt_feriados', inplace=True)

In [304]:
mergao

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,receita,categoria_loja,receita_minha_categoria,receita_trim1,receita_trim2,receita_trim3,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08,mes,ano,datetime,quarter,receita_media
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7851.879490
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7851.879490
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,1,7851.879490
3,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,2164.207949
4,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,2164.207949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,4,14337.009729
1655,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,4,14337.009729
1656,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7800.357442
1657,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7800.357442


In [305]:
feriados_df

,Unnamed: 0,cod_loja,mes,cnt_feriados
0,0,10,1,1.0
1,1,10,2,0.0
2,2,10,3,0.0
3,3,10,4,1.0
4,4,10,5,1.0
...,...,...,...,...
6631,6631,3684,8,0.0
6632,6632,3684,9,1.0
6633,6633,3684,10,1.0
6634,6634,3684,11,2.0


In [306]:
mergao_fer = mergao.merge(feriados_df, on=['cod_loja', 'mes'])
mergao_fer.drop(columns=['Unnamed: 0'], inplace=True)

In [307]:
mergao_fer

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,receita,categoria_loja,receita_minha_categoria,receita_trim1,receita_trim2,receita_trim3,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08,mes,ano,datetime,quarter,receita_media,cnt_feriados
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7851.879490,1.0
1,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7851.879490,0.0
2,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,0,50.00,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,6851.975507,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,1,7851.879490,0.0
3,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,2164.207949,1.0
4,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,41.00,4,0.116662,0.114995,26.345742,1.312612,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,84.3,77.5,100.0,100.0,63.5,78.8,2171.921852,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,2164.207949,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,4,14337.009729,0.0
1655,3675,4218202003002,4218202,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,40.74,4,0.133754,0.134264,20.027909,2.422305,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,92.5,100.0,100.0,94.2,96.6,7315.378882,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,3,2018,2018-03-01,4,14337.009729,0.0
1656,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,2018,2018-01-01,1,7800.357442,1.0
1657,3684,2917607003002,2917607,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,0,29.00,5,0.123648,0.113905,14.781510,1.803459,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,77.0,100.0,100.0,50.0,99.7,85.6,7129.139130,0,8639.382665,0.0,0.0,0.0,0,0,1,0,0,0,0,0,2,2018,2018-02-01,1,7800.357442,0.0


In [308]:
mergao_fer.drop(columns=str_cols, inplace=True)

In [309]:
submit_df = mergao_fer

In [310]:
submit_df

,cod_loja,cod_ap,cod_municipio,feature_03,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,categoria_loja,receita_minha_categoria,feature_05_FEATURE_05_VALUE_01,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08,mes,ano,quarter,receita_media,cnt_feriados
0,10,2933307005008,2933307,0,50.00,6,0.134654,0.131481,116.194058,2.913498,100.0,93.4,100.0,100.0,71.7,88.8,0,8639.382665,0,0,1,0,0,0,0,0,1,2018,1,7851.879490,1.0
1,10,2933307005008,2933307,0,50.00,6,0.134654,0.131481,116.194058,2.913498,100.0,93.4,100.0,100.0,71.7,88.8,0,8639.382665,0,0,1,0,0,0,0,0,2,2018,1,7851.879490,0.0
2,10,2933307005008,2933307,0,50.00,6,0.134654,0.131481,116.194058,2.913498,100.0,93.4,100.0,100.0,71.7,88.8,0,8639.382665,0,0,1,0,0,0,0,0,3,2018,1,7851.879490,0.0
3,13,2916401003002,2916401,0,41.00,4,0.116662,0.114995,26.345742,1.312612,84.3,77.5,100.0,100.0,63.5,78.8,0,8639.382665,0,0,1,0,0,0,0,0,1,2018,1,2164.207949,1.0
4,13,2916401003002,2916401,0,41.00,4,0.116662,0.114995,26.345742,1.312612,84.3,77.5,100.0,100.0,63.5,78.8,0,8639.382665,0,0,1,0,0,0,0,0,2,2018,1,2164.207949,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,3675,4218202003002,4218202,0,40.74,4,0.133754,0.134264,20.027909,2.422305,100.0,92.5,100.0,100.0,94.2,96.6,0,8639.382665,0,0,1,0,0,0,0,0,2,2018,4,14337.009729,0.0
1655,3675,4218202003002,4218202,0,40.74,4,0.133754,0.134264,20.027909,2.422305,100.0,92.5,100.0,100.0,94.2,96.6,0,8639.382665,0,0,1,0,0,0,0,0,3,2018,4,14337.009729,0.0
1656,3684,2917607003002,2917607,0,29.00,5,0.123648,0.113905,14.781510,1.803459,77.0,100.0,100.0,50.0,99.7,85.6,0,8639.382665,0,0,1,0,0,0,0,0,1,2018,1,7800.357442,1.0
1657,3684,2917607003002,2917607,0,29.00,5,0.123648,0.113905,14.781510,1.803459,77.0,100.0,100.0,50.0,99.7,85.6,0,8639.382665,0,0,1,0,0,0,0,0,2,2018,1,7800.357442,0.0


In [311]:
import catboost
model_ikeda = CatBoostRegressor()
model = CatBoostRegressor()

model_ikeda.load_model(zdata + "../modelo_catboost_otimizado_q234")
model.load_model(zdata + "../modelo_catboost_otimizado")

In [321]:
pred_normal = []
pred_ikeda = []
feats_in_order = ['cod_ap', 'cod_municipio', 'feature_03', 'feature_04', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
       'quarter', 'ano', 'mes', 'categoria_loja', 'receita_media',
       'cnt_feriados', 'receita_minha_categoria',
       'feature_05_FEATURE_05_VALUE_01', 'feature_05_FEATURE_05_VALUE_02',
       'feature_05_FEATURE_05_VALUE_03', 'feature_05_FEATURE_05_VALUE_04',
       'feature_05_FEATURE_05_VALUE_05', 'feature_05_FEATURE_05_VALUE_06',
       'feature_05_FEATURE_05_VALUE_07', 'feature_05_FEATURE_05_VALUE_08']

for cloja in mergao_fer.cod_loja.unique():
    test_index = (mergao_fer["cod_loja"] == cloja)
    
    yhat_normal = model.predict(mergao_fer[test_index].drop('cod_loja', axis=1, errors='ignore')[feats_in_order])
    pred_normal.append([cloja, np.mean(yhat_normal)])
    
    yhat_ikeda = model_ikeda.predict(mergao_fer[test_index].drop('cod_loja', axis=1, errors='ignore')[feats_in_order])
    pred_ikeda.append([cloja, np.mean(yhat_normal)])

In [324]:
pred_model_df = pd.DataFrame(pred_normal, columns=["cod_loja", "faturamento"])
pred_ikeda_df = pd.DataFrame(pred_ikeda, columns=["cod_loja", "faturamento"])

pred_model_df.to_csv("desafio_2_model.csv", index=False,sep=';')
pred_ikeda_df.to_csv("desafio_2_ikeda.csv", index=False,sep=';')